In [40]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [41]:

# Define dataset path
data_dir = "D:/Music_v3/Emotion_Based_Music_Player_with_Age_Preference/UTKFace"
images = []
labels = []


In [42]:
image_size = (64, 64)  # Reduce image size to save memory
batch_size = 16  # Reduce batch size to prevent memory overload

In [ ]:
for file in os.listdir(data_dir):
    if file.endswith(".jpg") and file.count("_") >= 2:
        parts = file.split("_")
        try:
            age, gender, race = int(parts[0]), int(parts[1]), int(parts[2])
            img_path = os.path.join(data_dir, file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, image_size)  # Resize to 64x64
            images.append(img)
            labels.append(gender)
        except ValueError:
            print(f"Skipping file: {file}")

In [44]:
# Convert lists to NumPy arrays
images = np.array(images, dtype=np.float32) / 255.0  # Normalize
labels = np.array(labels, dtype=np.uint8)


In [45]:
# Free up memory by limiting dataset size (optional)
max_samples = 10000  # Reduce dataset size to prevent MemoryError
images = images[:max_samples]
labels = labels[:max_samples]


In [46]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# Define CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification (Male/Female)
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Early stopping callback
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=batch_size, callbacks=[early_stopping])


In [ ]:
model.save("gender_classifier_model.h5")
print("Model saved successfully!")

In [50]:
# Testing the model
def predict_gender(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, image_size)
    img = np.array(img, dtype=np.float32) / 255.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    
    prediction = model.predict(img)
    gender = "Female" if prediction[0][0] > 0.5 else "Male"
    print(f"Predicted Gender: {gender}")
    return gender



In [ ]:
# Example usage
predict_gender("D:/Music_v3/pexels-olly-762020.jpg")

In [ ]:

# Evaluate model
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")